In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import *
from sklearn import metrics
from pymatreader import read_mat
import numpy as np
import json
import os

In [35]:
cwd = os.getcwd()
jsonfiledir = os.path.join(cwd, 'EEGData')
jsonfiles = []


for file in os.listdir(jsonfiledir): #only json files in directory
    print('loading', file)
    filepth = os.path.join(jsonfiledir, file)
    f = open(filepth)
    data = json.load(f)
    data['sessionID'] = file
    jsonfiles.append(data)
    f.close()

loading 15576668519182930000.json
loading 17957179669740114000.json
loading 20199147052280082000.json
loading 20720091866675330000.json
loading 30329014825927670000.json
loading 31908791274438530000.json
loading 33881536642638900000.json
loading 37476885619405180000.json
loading 40263762851668330000.json
loading 51647007780956774000.json
loading 65578173022607020000.json
loading 70773977546393580000.json
loading 70949146837861695000.json
loading 71762937516704770000.json
loading 74708274640981460000.json
loading 80012270501270370000.json
loading 92471859140639980000.json
loading 9257898488.json
loading 9283782737282763.json


In [36]:
for d in jsonfiles:
    print(d['sessionID'])
    print(len( d['channel1'] ))

15576668519182930000.json
123900
17957179669740114000.json
158496
20199147052280082000.json
139325
20720091866675330000.json
139600
30329014825927670000.json
139825
31908791274438530000.json
159248
33881536642638900000.json
139775
37476885619405180000.json
139664
40263762851668330000.json
120016
51647007780956774000.json
90112
65578173022607020000.json
48768
70773977546393580000.json
138975
70949146837861695000.json
131904
71762937516704770000.json
139200
74708274640981460000.json
139950
80012270501270370000.json
139950
92471859140639980000.json
139250
9257898488.json
139375
9283782737282763.json
139275


In [52]:
for d in jsonfiles:
    chandata = np.array((d['channel1'], \
                        d['channel2'], \
                        d['channel3'], \
                        d['channel4'], \
                        d['channel5'], \
                        d['channel6'], \
                        d['channel7'], \
                        d['channel8']))
    d['chandata'] = chandata
    d['chanavg'] = np.mean(chandata, axis=0)

In [67]:
jsonfiles[0]['chandata'].shape #chan x time
_t = jsonfiles[0]['chanavg']
len(_t)
_t[0], _t[123900 // 2], _t[-1]

(4.9846085790527646, -3.6827051198582392, 4.127897949370622)

In [55]:
NUM_POINTS = 120000
for d in jsonfiles:
    #interpolate to 120,000 points
    #d['chanavg']
    eeglen = len(d['chanavg'])
    x = [i * eeglen / NUM_POINTS for i in range(NUM_POINTS)] #x-coordinates to evaluate interp. values
    d['chanavginterp'] = np.interp(x, range(eeglen), d['chanavg'])

In [68]:
jsonfiles[0]['chanavginterp'].shape
_t = jsonfiles[0]['chanavginterp']
print(_e)
_t[0], _t[120000 // 2], _t[-1]

123900


(4.9846085790527646, -3.6827051198582392, 4.146616506650101)

In [78]:
S = 10 #10 songs
for d in jsonfiles:
    d['datasplit'] = []
    for i in range(S):
        s, e = i * NUM_POINTS // S, (i+1) * NUM_POINTS // S
        d['datasplit'].append(d['chanavginterp'][s:e])

In [89]:
_t = jsonfiles[0]['datasplit']
print( _t[0][-2] )
print( _t[0][-1] )
print( _t[1][0] )
print( _t[1][1] )

5.360310920501755
1.3892096040944408
-0.42941794673878064
-0.7861500054014877


In [90]:
#P = 19
#data = np.zeros(shape=(S*P, NUM_POINTS // S)) # S*P x NUM_POINTS = 10*19 x 120,000
#ratings = np.zeros(S*P)

In [93]:
#ind = 0
#for i in range(P):
#    for j in range(S):
#        data[ind] = jsonfiles[i]['datasplit'][j]
#        ind += 1

In [148]:
def getID(id):
    for entry in jsonfiles:
        if entry['sessionID'] == id:
            return entry
    return None

In [149]:
import csv
csvfile = os.path.join(cwd, 'UserSessionData.csv')
with open(csvfile, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count > 0:
            title = row[0]
            ratings = eval(row[4])
            for i, rating in enumerate(ratings):
                ratings[i] = int(rating)
            #print(ratings)
            titlejson = title + '.json'
            print(titlejson)
            print(idExists(titlejson))
            id = getID(titlejson)
            if id:
                id['ratings'] = ratings
        line_count += 1

33881536642638900000.json
True
80012270501270370000.json
True
74046263142686880000.json
False
15576668519182930000.json
True
17957179669740114000.json
True
74708274640981460000.json
True
30329014825927670000.json
True
37476885619405180000.json
True
70949146837861695000.json
True
19162543256201593000.json
False
92471859140639980000.json
True
40263762851668330000.json
True
71677240630889220000.json
False
31908791274438530000.json
True
51647007780956774000.json
True
40519162213428250000.json
False
65578173022607020000.json
True
15576668519182930000.json
True
20199147052280082000.json
True
44576528811972540000.json
False
26444285876842930000.json
False


In [154]:
jftrim = []
for x in jsonfiles:
    if 'ratings' in x.keys():
        jftrim.append(x)

In [156]:
len(jftrim)

14

In [157]:
S = 10
P = 14
NUM_POINTS = 120000
data = np.zeros(shape=(S*P, NUM_POINTS // S)) # S*P x NUM_POINTS = 10*19 x 120,000
ratings = np.zeros(S*P)

In [160]:
ind = 0
for i in range(P):
    for j in range(S):
        data[ind] = jftrim[i]['datasplit'][j]
        ind += 1

In [164]:
ind = 0
for i in range(P):
    print(jftrim[i]['sessionID'])
    rt = jftrim[i]['ratings']
    print(rt)
    assert(len(rt) == 10)
    for j in range(S):
        ratings[ind] = rt[j]
        ind += 1
print(ratings)

15576668519182930000.json
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
17957179669740114000.json
[2, 3, 3, 2, 1, 3, 2, 3, 2, 2]
20199147052280082000.json
[2, 2, 1, 2, 2, 2, 2, 2, 2, 2]
30329014825927670000.json
[3, 1, 1, 2, 2, 2, 2, 3, 1, 4]
31908791274438530000.json
[2, 3, 4, 2, 4, 2, 1, 1, 3, 4]
33881536642638900000.json
[4, 2, 2, 2, 2, 2, 2, 3, 4, 4]
37476885619405180000.json
[4, 3, 4, 2, 3, 4, 1, 3, 4, 1]
40263762851668330000.json
[3, 4, 2, 3, 2, 1, 2, 2, 3, 1]
51647007780956774000.json
[3, 4, 2, 2, 2, 2, 2, 3, 4, 1]
65578173022607020000.json
[3, 4, 3, 4, 4, 2, 2, 2, 2, 2]
70949146837861695000.json
[3, 3, 1, 4, 1, 3, 3, 4, 4, 4]
74708274640981460000.json
[2, 2, 2, 2, 2, 2, 2, 2, 3, 2]
80012270501270370000.json
[2, 4, 4, 2, 2, 1, 1, 2, 4, 3]
92471859140639980000.json
[2, 2, 2, 2, 2, 2, 2, 2, 2, 4]
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 2. 1. 3. 2. 3. 2. 2. 2. 2. 1. 2.
 2. 2. 2. 2. 2. 2. 3. 1. 1. 2. 2. 2. 2. 3. 1. 4. 2. 3. 4. 2. 4. 2. 1. 1.
 3. 4. 4. 2. 2. 2. 2. 2. 2. 3. 4. 4. 4. 3. 4. 2. 3. 4. 1

In [165]:
clf = RandomForestRegressor()
x_train, x_test, y_train, y_test = \
    train_test_split(data, ratings, test_size = 0.70, random_state = 5)
clf.fit(x_train, y_train)
y_pred_test = clf.predict(x_test)
diffs = []
for i in range(50):
    diffs.append( y_pred_test[i] - y_test[i] )
print(np.mean([x**2 for x in diffs]))

0.9702780000000001
